# Data Inverstigation and Processing

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when


# Initialize Spark session with configurations
spark = SparkSession.builder.config("spark.app.name", "Data Investigation") \
                         .config("spark.executor.memory", "8g") \
                         .config("spark.executor.cores", "4") \
                         .config("spark.driver.memory", "4g") \
                         .config("spark.sql.shuffle.partitions", "200") \
                         .getOrCreate()


# Path to the CSV file in Google Cloud Storage
csv_path = "gs://msca-bdp-student-gcs/Group5/Steam_data/steam_data_all.csv"

# Load the CSV file into a Spark DataFrame
df = spark.read.csv(csv_path, header=True, inferSchema=True)

# 1. Display the schema of the DataFrame
print("Schema of the DataFrame:")
df.printSchema()

# 2. Show the first few rows of the DataFrame
print("\nFirst Few Rows:")
df.show(3)

24/12/01 21:37:44 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


Schema of the DataFrame:
root
 |-- appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- positive: string (nullable = true)
 |-- negative: string (nullable = true)
 |-- owners: string (nullable = true)
 |-- average_forever: string (nullable = true)
 |-- average_2weeks: string (nullable = true)
 |-- median_forever: integer (nullable = true)
 |-- median_2weeks: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- initialprice: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- ccu: integer (nullable = true)
 |-- Afrikaans: integer (nullable = true)
 |-- Arabic: integer (nullable = true)
 |-- Armenian: integer (nullable = true)
 |-- Basque: integer (nullable = true)
 |-- Belarusian: integer (nullable = true)
 |-- Bulgarian: integer (nullable = true)
 |-- Catalan: integer (nullable = true)
 |-- Croatian: integer (nullable = true)
 |-- Czech: integer (nu

24/12/01 21:37:57 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-------------------+--------------------+--------+--------+--------------------+---------------+--------------+--------------+-------------+-----+------------+--------+-----+---------+------+--------+------+----------+---------+-------+--------+-----+------+-----+-------+--------+--------+-------+------+--------+--------+------+-----+------+-----+---------+---------+----------+-----+-------+--------+-------+------+------+-------+----------+-----+-----+---------+---------+-------------+-------+------+----------+-------------------+---------------------+--------+-------+-------+------------------+------+---------+-------+-----------------------+---------------+-------+-----+------+----+-------------------+-------+---------+----------+------------+------------------+---------------+--------------------------+----------------------+------------+---------------------------+------------------+---------------+--------------------+--------------------+------------

In [2]:
from pyspark.sql import functions as F
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

#number of partitions
df.rdd.getNumPartitions()
# From the partition distribution, it appears that partition #14 has significantly fewer records
# Repartition
df = df.repartition(30)
displaypartitions(df)

Partitions: 30


+-----------+-----+
|partitionId|count|
+-----------+-----+
|         13|  865|
|         14|  865|
|         15|  865|
|         16|  866|
|         28|  866|
|         26|  866|
|         27|  866|
|         12|  866|
|         17|  866|
|         20|  866|
|         23|  866|
|         19|  866|
|         22|  866|
|         24|  866|
|         25|  866|
|         21|  866|
|         29|  866|
|         11|  866|
|          0|  866|
|         18|  866|
|          1|  867|
|          3|  867|
|          6|  867|
|          8|  867|
|         10|  867|
|          4|  867|
|          9|  867|
|          7|  867|
|          5|  867|
|          2|  867|
+-----------+-----+



In [3]:
# Columns to convert
columns_to_convert = ['is_free', 'windows', 'mac', 'linux']

# Convert specified columns to integer
for column in columns_to_convert:
    df = df.withColumn(column, col(column).cast('int'))

# Display the DataFrame
df.printSchema()


root
 |-- appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- positive: string (nullable = true)
 |-- negative: string (nullable = true)
 |-- owners: string (nullable = true)
 |-- average_forever: string (nullable = true)
 |-- average_2weeks: string (nullable = true)
 |-- median_forever: integer (nullable = true)
 |-- median_2weeks: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- initialprice: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- ccu: integer (nullable = true)
 |-- Afrikaans: integer (nullable = true)
 |-- Arabic: integer (nullable = true)
 |-- Armenian: integer (nullable = true)
 |-- Basque: integer (nullable = true)
 |-- Belarusian: integer (nullable = true)
 |-- Bulgarian: integer (nullable = true)
 |-- Catalan: integer (nullable = true)
 |-- Croatian: integer (nullable = true)
 |-- Czech: integer (nullable = true)
 |-- Danis

In [4]:
#clean the name
import re

# Replace characters that are not letters or digits with underscores
def clean_column_name(col_name):
    return re.sub(r'[^a-zA-Z0-9]', '_', col_name)

# Create a dictionary for mapping
cleaned_columns = {c: clean_column_name(c) for c in df.columns}

# Rename columns in DF using cleaned column names
for original_col, cleaned_col in cleaned_columns.items():
    df = df.withColumnRenamed(original_col, cleaned_col)


In [5]:
record_count = df.count()
print(f"Total Records: {record_count}")

Total Records: 25987


### Change to right type

In [6]:
from pyspark.sql.functions import col, when

# Separate columns into numeric-related and dummy columns
numeric_columns = [
    "positive", "negative", "average_forever", "average_2weeks", 
    "median_forever", "median_2weeks", "price", "initialprice", 
    "discount", "ccu","recommendations","score"
]

dummy_columns = [
    "Afrikaans", "Arabic", "Armenian", "Basque", "Belarusian", 
    "Bulgarian", "Catalan", "Croatian", "Czech", "Danish", "Dutch", 
    "English", "Estonian", "Filipino", "Finnish", "French", 
    "Galician", "Georgian", "German", "Greek", "Hebrew", "Hindi", 
    "Hungarian", "Icelandic", "Indonesian", "Irish", "Italian", 
    "Japanese", "Kannada", "Kazakh", "Korean", "Latvian", 
    "Lithuanian", "Malay", "Maori","Mongolian",
    "Norwegian",
    "Not supported",
    "Persian",
    "Polish",
    "Portuguese",
    "Portuguese - Brazil",
    "Portuguese - Portugal",
    "Romanian",
    "Russian",
    "Serbian",
    "Simplified Chinese",
    "Slovak",
    "Slovenian",
    "Spanish",
    "Spanish - Latin America",
    "Spanish - Spain",
    "Swedish",
    "Tamil",
    "Telugu",
    "Thai",
    "Traditional Chinese",
    "Turkish",
    "Ukrainian",
    "Vietnamese"
]


# 1. Convert numeric-related columns
for column in numeric_columns:
    df = df.withColumn(
        column,
        when(col(column).rlike("^[0-9]+$"), col(column))  # Retain only valid numeric strings
        .otherwise(None)  # Replace invalid entries with null
        .cast("int")  # Cast to integer
    )

# 2. Convert dummy columns
#for column in dummy_columns:
    #df = df.withColumn(
        #column,
        #when(col(column).rlike("^[01]$"), col(column))  # Retain only valid binary values (0 or 1)
        #.otherwise(0)  # Replace invalid entries with 0 (default for binary)
        #.cast("int")  # Cast to integer
    #)

# Verify the schema after transformation
df.printSchema()


root
 |-- appid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- positive: integer (nullable = true)
 |-- negative: integer (nullable = true)
 |-- owners: string (nullable = true)
 |-- average_forever: integer (nullable = true)
 |-- average_2weeks: integer (nullable = true)
 |-- median_forever: integer (nullable = true)
 |-- median_2weeks: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- initialprice: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- ccu: integer (nullable = true)
 |-- Afrikaans: integer (nullable = true)
 |-- Arabic: integer (nullable = true)
 |-- Armenian: integer (nullable = true)
 |-- Basque: integer (nullable = true)
 |-- Belarusian: integer (nullable = true)
 |-- Bulgarian: integer (nullable = true)
 |-- Catalan: integer (nullable = true)
 |-- Croatian: integer (nullable = true)
 |-- Czech: integer (nullable = true)
 |-- D

### Clean categorical columns

In [7]:
categorical_columns = [col for col, dtype in df.dtypes if dtype == 'string']
for col in categorical_columns:
    print(col)

name
developer
publisher
owners
release_date


In [8]:
#Drop the unnecessary columns
df = df.drop('developer', 'publisher', 'release_date')


In [9]:
from pyspark.sql.functions import col, when, count
#Deal with null value
df.select(
    [count(when(col(c).isNull() | (col(c) == ""), c)).alias(c) for c in ['appid', 'name', 'owners']]
).show()
df = df.na.drop(subset=['appid', 'name', 'owners'])

+-----+----+------+
|appid|name|owners|
+-----+----+------+
|    0|  57|     0|
+-----+----+------+



In [10]:
record_count = df.count()
print(f"Total Records: {record_count}")

Total Records: 25930


In [11]:
#Deal with duplicates
duplicates = df.groupBy('appid').count().filter(col('count') > 1)
duplicates.show()
df = df.dropDuplicates(['appid'])

+------+-----+
| appid|count|
+------+-----+
| 32460|    2|
| 34330|    2|
| 56400|    2|
| 22330|    2|
| 38480|    2|
|311210|    2|
| 33220|    2|
|    80|    2|
|  9200|    2|
| 42680|    2|
+------+-----+



In [12]:
record_count = df.count()
print(f"Total Records: {record_count}")

Total Records: 25920


### Clean numerical Variables

In [13]:
# Display summary statistics for numerical columns
numerical_columns = [
    'positive', 'negative', 'average_forever', 'average_2weeks', 
    'median_forever', 'median_2weeks', 'price', 'initialprice', 
    'discount', 'ccu',"recommendations","score"
]
df.select(numerical_columns).describe().show()


+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-----+
|summary|         positive|          negative|   average_forever|   average_2weeks|    median_forever|     median_2weeks|             price|      initialprice|          discount|               ccu|recommendations|score|
+-------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+-----+
|  count|            25917|             25919|             25918|            25919|             25920|             25920|             25920|             25920|             25920|             25920|              0|    0|
|   mean|3969.599452097079| 640.1754697326286| 76.49915116907168|3.058567074347004| 56.72523148148148| 3.182793209876543

In [14]:
#Deal with missing Value
df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in numerical_columns]
).show()

# Fill missing values with the mean for each column
df = df.fillna(0, subset=numerical_columns)

+--------+--------+---------------+--------------+--------------+-------------+-----+------------+--------+---+---------------+-----+
|positive|negative|average_forever|average_2weeks|median_forever|median_2weeks|price|initialprice|discount|ccu|recommendations|score|
+--------+--------+---------------+--------------+--------------+-------------+-----+------------+--------+---+---------------+-----+
|       3|       1|              2|             1|             0|            0|    0|           0|       0|  0|          25920|25920|
+--------+--------+---------------+--------------+--------------+-------------+-----+------------+--------+---+---------------+-----+



In [15]:
# Identify and filter outliers using approxQuantile for each column
# Calculate and print outlier counts for each column
for column in numerical_columns:
    # Calculate Q1 and Q3 for the column
    q1, q3 = df.approxQuantile(column, [0.10, 0.90], 0.01)  # Approximate quantiles
    iqr = q3 - q1  # Interquartile range
    lower_bound = q1 - 1.5 * iqr  # Lower bound
    upper_bound = q3 + 1.5 * iqr  # Upper bound
    
    # Count outliers below the lower bound
    lower_outliers = df.filter(col(column) < lower_bound).count()
    # Count outliers above the upper bound
    upper_outliers = df.filter(col(column) > upper_bound).count()
    # Total outliers
    total_outliers = lower_outliers + upper_outliers

    print(f"Column: {column}")
    print(f"  Lower Outliers: {lower_outliers}")
    print(f"  Upper Outliers: {upper_outliers}")
    print(f"  Total Outliers: {total_outliers}")
    print("-" * 40)


Column: positive
  Lower Outliers: 0
  Upper Outliers: 1508
  Total Outliers: 1508
----------------------------------------


Column: negative
  Lower Outliers: 0
  Upper Outliers: 1410
  Total Outliers: 1410
----------------------------------------


Column: average_forever
  Lower Outliers: 0
  Upper Outliers: 1903
  Total Outliers: 1903
----------------------------------------


Column: average_2weeks
  Lower Outliers: 0
  Upper Outliers: 177
  Total Outliers: 177
----------------------------------------


Column: median_forever
  Lower Outliers: 0
  Upper Outliers: 1902
  Total Outliers: 1902
----------------------------------------


...

Column: initialprice
  Lower Outliers: 0
  Upper Outliers: 155
  Total Outliers: 155
----------------------------------------


Column: discount
  Lower Outliers: 0
  Upper Outliers: 3
  Total Outliers: 3
----------------------------------------


Column: ccu
  Lower Outliers: 0
  Upper Outliers: 1856
  Total Outliers: 1856
----------------------------------------


Column: recommendations
  Lower Outliers: 0
  Upper Outliers: 0
  Total Outliers: 0
----------------------------------------


Column: score
  Lower Outliers: 0
  Upper Outliers: 0
  Total Outliers: 0
----------------------------------------


### Clean Dummy Variables

In [16]:
dummy_column = [
    'Afrikaans', 'Arabic', 'Armenian', 'Basque', 'Belarusian', 'Bulgarian', 
    'Catalan', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 
    'Estonian', 'Filipino', 'Finnish', 'French', 'Galician', 'Georgian', 
    'German', 'Greek', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 
    'Indonesian', 'Irish', 'Italian', 'Japanese', 'Kannada', 'Kazakh', 
    'Korean', 'Latvian', 'Lithuanian', 'Malay', 'Maori', 'Mongolian', 
    'Norwegian', 'Not_supported', 'Persian', 'Polish', 'Portuguese', 
    'Portuguese___Brazil', 'Portuguese___Portugal', 'Romanian', 'Russian', 
    'Serbian', 'Simplified_Chinese', 'Slovak', 'Slovenian', 'Spanish', 
    'Spanish___Latin_America', 'Spanish___Spain', 'Swedish', 'Tamil', 
    'Telugu', 'Thai', 'Traditional_Chinese', 'Turkish', 'Ukrainian', 
    'Vietnamese'
] + [col for col in df.columns if col.startswith('genre_')] + [col for col in df.columns if col.startswith('tag_')]

dummy_columns = [c for c in dummy_column if c in df.columns]


In [17]:
#deal with the missing value
df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in dummy_columns]
).show()


24/12/01 21:43:41 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1106.3 KiB


+---------+------+--------+------+----------+---------+-------+--------+-----+------+-----+-------+--------+--------+-------+------+--------+--------+------+-----+------+-----+---------+---------+----------+-----+-------+--------+-------+------+------+-------+----------+-----+-----+---------+---------+-------------+-------+------+----------+-------------------+---------------------+--------+-------+-------+------------------+------+---------+-------+-----------------------+---------------+-------+-----+------+----+-------------------+-------+---------+----------+------------+------------------+---------------+--------------------------+----------------------+------------+---------------------------+------------------+---------------+--------------------+--------------------+----------------------+-----------+---------------------------+-------------------+---------+------------+----------------+-----------------------+------------+--------------+---------------+----------------------+-

#### No missing value

#Validate Dummy variables
for column in dummy_columns:
    df = df.withColumn(
        column,
        when(col(column).rlike("^[01]$"), col(column))  # Retain valid 0 or 1
        .otherwise(0)  # Replace invalid or missing values with 0
        .cast("int")  # Ensure integer type
    )

In [ ]:
from pyspark.sql import functions as F

# Step 1: Define columns for languages, tags, and genres
language_columns = ["Afrikaans", "Arabic", "Armenian", "Basque", "Belarusian", 
    "Bulgarian", "Catalan", "Croatian", "Czech", "Danish", "Dutch", 
    "English", "Estonian", "Filipino", "Finnish", "French", 
    "Galician", "Georgian", "German", "Greek", "Hebrew", "Hindi", 
    "Hungarian", "Icelandic", "Indonesian", "Irish", "Italian", 
    "Japanese", "Kannada", "Kazakh", "Korean", "Latvian", 
    "Lithuanian", "Malay", "Maori", "Mongolian",
    "Norwegian",
    "Not supported",
    "Persian",
    "Polish",
    "Portuguese",
    "Portuguese - Brazil",
    "Portuguese - Portugal",
    "Romanian",
    "Russian",
    "Serbian",
    "Simplified Chinese",
    "Slovak",
    "Slovenian",
    "Spanish",
    "Spanish - Latin America",
    "Spanish - Spain",
    "Swedish",
    "Tamil",
    "Telugu",
    "Thai",
    "Traditional Chinese",
    "Turkish",
    "Ukrainian",
    "Vietnamese"]

tag_columns = [col for col in df.columns if col.startswith('tag_')]
genre_columns = [col for col in df.columns if col.startswith('genre_')]

def compute_and_print_counts(df, column_list, feature_name):
    """
    Function to compute and print counts of different features sorted in ascending order.
    :param df: PySpark DataFrame
    :param column_list: List of column names to compute the totals
    :param feature_name: Name of the feature type (e.g., Language, Tag, Genre)
    """
    from pyspark.sql import functions as F

    # Sum each column, handling special characters with F.col()
    sum_df = (
        df.select(
            *[F.sum(F.col(f"`{c}`")).alias(c) for c in column_list]
        ).toPandas()
        .T.reset_index()
    )
    sum_df.columns = ['Feature', 'Total']

    # Sort by 'Total' in ascending order
    sum_df = sum_df.sort_values(by='Total', ascending=True)

    # Display all counts
    print(f"\nCounts of all {feature_name} sorted in ascending order:")
    print(sum_df)


# Compute and print counts for languages, tags, and genres
compute_and_print_counts(df, language_columns, "Languages")
compute_and_print_counts(df, tag_columns, "Tags")
compute_and_print_counts(df, genre_columns, "Genres")



# Add rate ratio columns

In [ ]:
from pyspark.sql.functions import lit
import matplotlib.pyplot as plt

df = df.withColumn("number_of_reviews", (col("positive") + col("negative")))
df = df.withColumn("positive_ratio", col("positive") / (col("positive") + col("negative")))


df = df.withColumn(
    "tag",
    when((col("positive_ratio") >= 0.95) & (col("number_of_reviews") >= 500), lit("Overwhelmingly Positive"))
    .when((col("positive_ratio") >= 0.8) & (col("number_of_reviews") >= 50), lit("Very Positive"))
    .when((col("positive_ratio") >= 0.7), lit("Mostly Positive"))
    .when((col("positive_ratio") >= 0.4), lit("Mixed"))
    .when((col("positive_ratio") >= 0.2), lit("Mostly Negative"))
    .when((col("positive_ratio") >= 0.0), lit("Negative"))
    .otherwise(lit("Unknown"))
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

tag_counts = df.groupBy("tag").count().orderBy("count", ascending=False)

# Convert to Pandas DataFrame for visualization
tag_counts_pd = tag_counts.toPandas()

# Define colors for each tag
tag_colors = {
    "Overwhelmingly Positive": "darkgreen",
    "Very Positive": "green",
    "Mostly Positive": "yellowgreen",
    "Mixed": "gold",
    "Mostly Negative": "orange",
    "Negative": "red",
    "Unknown": "gray",
}

# Plot the data
plt.figure(figsize=(10, 6))
bars = plt.bar(
    tag_counts_pd["tag"], 
    tag_counts_pd["count"], 
    color=[tag_colors[tag] for tag in tag_counts_pd["tag"]], 
    edgecolor="black"
)

# Add the count numbers on top of the bars
for bar, count in zip(bars, tag_counts_pd["count"]):
    plt.text(
        bar.get_x() + bar.get_width() / 2, 
        bar.get_height() + 1, 
        str(count), 
        ha="center", 
        va="bottom", 
        fontsize=10
    )

# Step 5: Customize the plot
plt.title("Tag Distribution with Counts", fontsize=16)
plt.xlabel("Tag", fontsize=14)
plt.ylabel("Count", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
from pyspark.sql import functions as F

# Step 1: Filter out rows with null values
df = df.filter(F.col("positive_ratio").isNotNull() & F.col("number_of_reviews").isNotNull())

# Step 2: Define Positive % ranges and Review Categories
positive_ranges = [
    (0.95, 1.0, "95%-100%"),
    (0.9, 0.95, "90%-94%"),
    (0.85, 0.9, "85%-89%"),
    (0.8, 0.85, "80%-84%"),
    (0.75, 0.8, "75%-79%"),
    (0.7, 0.75, "70%-74%"),
    (0.4, 0.7, "40%-69% (Mixed)"),
    (0.2, 0.4, "20%-39% (Mostly Negative)"),
    (0.0, 0.2, "0%-19% (Negative)")
]

review_categories = [
    (0, 49, "10-49 reviews"),
    (50, 499, "50-499 reviews"),
    (500, float("inf"), "500+ reviews")
]

# Step 3: Updated UDFs without handling "Unknown"
def assign_positive_range(positive_ratio):
    for lower, upper, label in positive_ranges:
        if lower <= positive_ratio <= upper:
            return label

def assign_review_category(number_of_reviews):
    for lower, upper, label in review_categories:
        if lower <= number_of_reviews <= upper:
            return label

assign_positive_range_udf = F.udf(assign_positive_range)
assign_review_category_udf = F.udf(assign_review_category)

# Step 4: Apply the UDFs
df = df.withColumn("positive_range", assign_positive_range_udf(F.col("positive_ratio")))
df = df.withColumn("review_category", assign_review_category_udf(F.col("number_of_reviews")))

# Step 5: Group data by positive_range and review_category
table_data = df.groupBy("positive_range", "review_category").count().toPandas()

# Prepare the grid
positive_labels = [label for _, _, label in positive_ranges]
review_labels = [label for _, _, label in review_categories]
grid_data = np.zeros((len(positive_labels), len(review_labels)))

# Populate the grid with counts
for _, row in table_data.iterrows():
    i = positive_labels.index(row["positive_range"])
    j = review_labels.index(row["review_category"])
    grid_data[i, j] = row["count"]

# Step 6: Define colors for the grid
colors = {
    "95%-100%": "darkgreen",
    "90%-94%": "green",
    "85%-89%": "yellowgreen",
    "80%-84%": "gold",
    "75%-79%": "orange",
    "70%-74%": "lightcoral",
    "40%-69% (Mixed)": "khaki",
    "20%-39% (Mostly Negative)": "salmon",
    "0%-19% (Negative)": "red"
}

# Step 7: Plot the table graph
fig, ax = plt.subplots(figsize=(12, 8))
for i in range(len(positive_labels)):
    for j in range(len(review_labels)):
        value = grid_data[i, j]
        color = colors.get(positive_labels[i], "white")
        ax.add_patch(
            plt.Rectangle((j, i), 1, 1, facecolor=color, edgecolor="black")
        )
        ax.text(
            j + 0.5, i + 0.5, int(value), 
            ha="center", va="center", fontsize=10, color="black"
        )

# Add labels and titles
ax.set_xticks(np.arange(len(review_labels)) + 0.5)
ax.set_xticklabels(review_labels, fontsize=12, rotation=45)
ax.set_yticks(np.arange(len(positive_labels)) + 0.5)
ax.set_yticklabels(positive_labels, fontsize=12)
ax.set_xlim(0, len(review_labels))
ax.set_ylim(0, len(positive_labels))
ax.set_title("Tag Distribution by Positive % and Review Count", fontsize=16)

plt.tight_layout()
plt.show()

# Keep the necessary Data

In [ ]:
all_columns = df.columns
columns_to_keep = [
    'appid', 'name', 'owners', "positive", "negative", "average_forever", 
    "average_2weeks", "median_forever", "median_2weeks", "price", "initialprice", 
    "discount", "ccu", "recommendations", "score","number_of_reviews","positive_ratio",
    "tag","positive_range","review_category",
    "English", "German", 
    "French", "Spanish - Spain", "Simplified Chinese", "Russian", "Japanese", 
    "Italian", "Portuguese — Brazil", "Korean", "Traditional Chinese", "Polish", 
    "Turkish", "Spanish - Latin America", "Portuguese — Portugal", "Dutch", 
    "Czech", "Hungarian", "Swedish", "Ukrainian", "Arabic", "Danish", "Thai", 
    "Finnish", "Norwegian", "tag_Singleplayer", "tag_Adventure", "tag_Multiplayer", 
    "tag_Indie", "tag_RPG", "tag_Simulation", "tag_Strategy", "tag_Casual", 
    "tag_Open World", "tag_Atmospheric", "tag_Story Rich", "tag_First-Person", 
    "tag_2D", "tag_Shooter", "tag_Exploration", "tag_Co-op", "tag_Survival", 
    "tag_FPS", "tag_Free to Play", "tag_Sandbox", "tag_Horror", 
    "tag_Online Co-Op", "tag_Funny", "tag_Puzzle", "tag_3D", "tag_PvP", 
    "tag_Pixel Graphics", "tag_Fantasy", "tag_Cute", "tag_Action", 
    "tag_Singleplayer", "tag_Adventure", "tag_Multiplayer", "tag_Indie", 
    "tag_RPG", "tag_Simulation", "tag_Strategy"
]

columns_to_drop = list(set(all_columns) - set(columns_to_keep))

for col in columns_to_drop:
    df = df.drop(col)

# Verify the schema
df.printSchema()


# Store the data

In [ ]:
# Divide the DataFrame based on the price condition
free_df = df.filter(df["price"] == 0)
paid_df = df.filter(df["price"] != 0)